## Prep

In [ ]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


# -------------- commutation matrix --------------
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]

# -------------- Gram matrix --------------
def gram(A: np.ndarray) -> np.ndarray:
    return np.matmul(np.transpose(A), A)

# -------------- matrix A(Sigma) --------------
def create_A_Sigma(cov: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))

def create_A_Sigma_hard(cov: np.ndarray, p: int, index: int) -> np.ndarray:
    identity = np.eye(p)
    identity[index, index] = 0
    if p == index:
        cov = np.concatenate((cov, np.zeros((1, p-1))), axis=0)
        cov = np.concatenate((cov, np.zeros((p, 1))), axis=1)
    else:
        cov = np.insert(cov, index, 0, axis=0)
        cov = np.insert(cov, index, 0, axis=1)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))



# -------------- matrix A (intervention) --------------
def create_A_shift(cov: np.ndarray, mean_int: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    A_1 = create_A_Sigma(cov, p)
    temp = np.kron(mean_int, identity)
    return np.vstack((A_1, temp))

def create_A_hard(cov: np.ndarray, cov_int:np.ndarray, mean_int: np.ndarray, p: int, index: int) -> np.ndarray:
    identity = np.eye(p)
    identity[index, index] = 0
    mean_int = np.insert(mean_int, index, hard_param)
    A_1 = create_A_Sigma(cov, p)
    A_2 = create_A_Sigma_hard(cov_int, p-1, index)
    temp = np.kron(mean_int, identity)
    return np.vstack((A_1, A_2, temp))

# -------------- irrepresentability condition --------------
def irrep_cond(A: np.ndarray, support: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], support] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(support, support)])
        return np.linalg.norm(np.matmul(matrix_1, matrix_2), ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

# -------------- weak irrepresentability condition --------------
def weak_irrep_cond(M: np.ndarray, A: np.ndarray, support: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], support] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(support, support)])
        sign_vec = np.sign(M.flatten(order="F"))[support]
        return np.linalg.norm(np.matmul(matrix_1, matrix_2) @ sign_vec, ord=np.inf)
    except np.linalg.LinAlgError:
        return -1




num_exp = 100000 # number of true matrices per case
volatility_param = 2
shift_param = 2
hard_param = 2
reg_param = 0.001

## Experiments

### Ex. 0
<img src="Graphics/graph40.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.

### Ex. 1
<img src="Graphics/graph41.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 7, 8])
supp_compl = np.array([2, 5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_1 = ex_nonsimple_1.copy()
ex_nonsimple_shift_1 = ex_nonsimple_1.copy()

for row in ex_nonsimple_1.itertuples():
    row = row.Index
    true_M = ex_nonsimple_1.iloc[row, 0]

    intervention_index = ex_nonsimple_1.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_1.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_1.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_1 = ex_nonsimple_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_1 = ex_nonsimple_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_1 = ex_nonsimple_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_1.to_pickle("Experiments/irrep_nonsimple_1.pkl")
ex_nonsimple_hard_1.to_pickle("Experiments/irrep_nonsimple_hard_1.pkl")
ex_nonsimple_shift_1.to_pickle("Experiments/irrep_nonsimple_shift_1.pkl")

### Ex. 2
<img src="Graphics/graph42.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [2, 0], [2, 1]])
support = np.array([0, 1, 3, 4, 5, 8])
supp_compl = np.array([2, 6, 7])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_2 = ex_nonsimple_2.copy()
ex_nonsimple_shift_2 = ex_nonsimple_2.copy()

for row in ex_nonsimple_2.itertuples():
    row = row.Index
    true_M = ex_nonsimple_2.iloc[row, 0]

    intervention_index = ex_nonsimple_2.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_2.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_2.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_2.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_2.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_2.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_2.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_2 = ex_nonsimple_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_2 = ex_nonsimple_hard_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_2 = ex_nonsimple_shift_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_2.to_pickle("Experiments/irrep_nonsimple_2.pkl")
ex_nonsimple_hard_2.to_pickle("Experiments/irrep_nonsimple_hard_2.pkl")
ex_nonsimple_shift_2.to_pickle("Experiments/irrep_nonsimple_shift_2.pkl")

### Ex. 3
<img src="Graphics/graph43.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 2]])
support = np.array([0, 1, 3, 4, 6, 7, 8])
supp_compl = np.array([2, 5])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_3 = ex_nonsimple_3.copy()
ex_nonsimple_shift_3 = ex_nonsimple_3.copy()

for row in ex_nonsimple_3.itertuples():
    row = row.Index
    true_M = ex_nonsimple_3.iloc[row, 0]

    intervention_index = ex_nonsimple_3.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_3.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_3.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_3.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_3.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_3.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_3.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_3 = ex_nonsimple_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_3 = ex_nonsimple_hard_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_3 = ex_nonsimple_shift_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_3.to_pickle("Experiments/irrep_nonsimple_3.pkl")
ex_nonsimple_hard_3.to_pickle("Experiments/irrep_nonsimple_hard_3.pkl")
ex_nonsimple_shift_3.to_pickle("Experiments/irrep_nonsimple_shift_3.pkl")

### Ex. 4
<img src="Graphics/graph44.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[1, 2], [2, 0]])
support = np.array([0, 1, 2, 3, 4, 7, 8])
supp_compl = np.array([5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_4 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_4 = ex_nonsimple_4.copy()
ex_nonsimple_shift_4 = ex_nonsimple_4.copy()

for row in ex_nonsimple_4.itertuples():
    row = row.Index
    true_M = ex_nonsimple_4.iloc[row, 0]

    intervention_index = ex_nonsimple_4.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_4.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_4.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_4.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_4.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_4.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_4.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_4 = ex_nonsimple_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_4 = ex_nonsimple_hard_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_4 = ex_nonsimple_shift_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_4.to_pickle("Experiments/irrep_nonsimple_4.pkl")
ex_nonsimple_hard_4.to_pickle("Experiments/irrep_nonsimple_hard_4.pkl")
ex_nonsimple_shift_4.to_pickle("Experiments/irrep_nonsimple_shift_4.pkl")

### Ex. 5
<img src="Graphics/graph45.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[2, 0], [2, 1]])
support = np.array([0, 1, 2, 3, 4, 5, 8])
supp_compl = np.array([6, 7])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_5 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_5 = ex_nonsimple_5.copy()
ex_nonsimple_shift_5 = ex_nonsimple_5.copy()

for row in ex_nonsimple_5.itertuples():
    row = row.Index
    true_M = ex_nonsimple_5.iloc[row, 0]

    intervention_index = ex_nonsimple_5.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_5.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_5.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_5.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_5.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_5.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_5.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_5 = ex_nonsimple_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_5 = ex_nonsimple_hard_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_5 = ex_nonsimple_shift_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_5.to_pickle("Experiments/irrep_nonsimple_5.pkl")
ex_nonsimple_hard_5.to_pickle("Experiments/irrep_nonsimple_hard_5.pkl")
ex_nonsimple_shift_5.to_pickle("Experiments/irrep_nonsimple_shift_5.pkl")

### Ex. 6
<img src="Graphics/graph46.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 5, 7, 8])
supp_compl = np.array([2, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_6 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_6 = ex_nonsimple_6.copy()
ex_nonsimple_shift_6 = ex_nonsimple_6.copy()

for row in ex_nonsimple_6.itertuples():
    row = row.Index
    true_M = ex_nonsimple_6.iloc[row, 0]

    intervention_index = ex_nonsimple_6.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_6.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_6.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_6.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_6.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_6.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_6.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_6 = ex_nonsimple_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_6 = ex_nonsimple_hard_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_6 = ex_nonsimple_shift_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_6.to_pickle("Experiments/irrep_nonsimple_6.pkl")
ex_nonsimple_hard_6.to_pickle("Experiments/irrep_nonsimple_hard_6.pkl")
ex_nonsimple_shift_6.to_pickle("Experiments/irrep_nonsimple_shift_6.pkl")

### Ex. 7
<img src="Graphics/graph47.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2]])
support = np.array([0, 1, 3, 4, 5, 6, 7, 8])
supp_compl = np.array([2])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_nonsimple_7 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_7 = ex_nonsimple_7.copy()
ex_nonsimple_shift_7 = ex_nonsimple_7.copy()

for row in ex_nonsimple_7.itertuples():
    row = row.Index
    true_M = ex_nonsimple_7.iloc[row, 0]

    intervention_index = ex_nonsimple_7.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_nonsimple_7.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_nonsimple_7.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_nonsimple_shift_7.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_nonsimple_shift_7.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_nonsimple_hard_7.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_nonsimple_hard_7.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_nonsimple_7 = ex_nonsimple_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_hard_7 = ex_nonsimple_hard_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_shift_7 = ex_nonsimple_shift_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_nonsimple_7.to_pickle("Experiments/irrep_nonsimple_7.pkl")
ex_nonsimple_hard_7.to_pickle("Experiments/irrep_nonsimple_hard_7.pkl")
ex_nonsimple_shift_7.to_pickle("Experiments/irrep_nonsimple_shift_7.pkl")

### Ex. 0
<img src="Graphics/graph48.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.